In [1]:
!pip install twarc

     ---------------------------------------- 60.1/60.1 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 110.0/110.0 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 8.8 MB/s eta 0:00:00


In [4]:
!pip install pymongo

     ------------------------------------- 382.5/382.5 kB 12.0 MB/s eta 0:00:00
     ------------------------------------- 283.7/283.7 kB 18.2 MB/s eta 0:00:00


In [7]:
from twarc import Twarc2, expansions
from __future__ import print_function
import json
from pymongo import MongoClient
import time
import os
import sys

# Replace your bearer token below
client = Twarc2(bearer_token="AAAAAAAAAAAAAAAAAAAAAMPMVgEAAAAARPMOg23iiV6iFggcUrCpP6iAfXc%3Dtc9b3IXwH2BPO2edlfomaCaYPJE88udPvAkI49RkqWHVysT1r9")

# connect to local mongodb
# assuming you have mongoDB installed locally, and you want to input tweets onto a databse called tweetsDB_v2
MONGO_HOST= 'mongodb://localhost:27017/solar_tweetsDB_v2'  
mongo_client = MongoClient(MONGO_HOST)

def main():
    # Remove existing rules
    existing_rules = client.get_stream_rules()
    if 'data' in existing_rules and len(existing_rules['data']) > 0:
        rule_ids = [rule['id'] for rule in existing_rules['data']]
        client.delete_stream_rule_ids(rule_ids)

    # Add new rules (keywords you want to conduct search)
    # Replace the rules below with the ones that you want to add as discussed in module 5
    new_rules = [
        {"value": "#solarenergy OR #solarpanels lang:en"}, #tweets that includes either hashtags and in English
        {"value": "solar energy OR solar panels OR renewable energy"}   # tweets that include climate change
    ]
   
    added_rules = client.add_stream_rules(rules=new_rules)
    
    # Connect to the filtered stream
    for count, result in enumerate(client.stream()):
        # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
        # so we use expansions.flatten to get all the information in a single JSON
        tweet = expansions.flatten(result)
        
        # extrac full twitter object to the console
        
        tweet1=json.dumps(tweet)
        
        # covert string to json object
        
        tweet2=json.loads(tweet1)        
        
        # print the time the tweet is created on console
        
        print(tweet2[0]['created_at'])
        
                         
        # Use tweetsDB_v2 database. If it doesn't exist, it will be created.
        db = mongo_client.solar_tweetsDB_v2
        
        #insert tweets into climate_change collection
        #if climate_change doesn't exist, it will be created.
        db.climate_change.insert(tweet2)
                
        # Replace with the desired number of Tweets
        if count > 1000000:
            time.sleep(60)

    # Delete the rules once you have collected the desired number of Tweets
    rule_ids = [rule['id'] for rule in added_rules['data']]
    client.delete_stream_rule_ids(rule_ids)

if __name__ == "__main__":
    main()

2023-02-28T14:57:36.000Z


TypeError: 'Collection' object is not callable. If you meant to call the 'insert' method on a 'Collection' object it is failing because no such method exists.